# TFrecord 생성파일

TFrecord 파일을 생성하기 위해서는 크게 2가지 있어야 함
1. 라벨과 바운딩박스 정보가 포함된 csv파일(다른확장자도 가능)
2. 클래스 분류에 따른 pbtxt 파일

2가지 만들었다면 4번 항목으로 ㄱㄱ

In [5]:
import pylab
import imageio
import cv2
import os
from glob import glob
from tqdm import tqdm,tnrange
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

## 1. Frame extract

In [ ]:
#train

for i in range(0,6):
    print(f'Start number : {i}')
    cam_num = i # 0부터 5까지 있다
    
    #추후 경로 변경 시수정이 필요한 사항 
    cam_dir = f'/home/workspace/data/.train/.task154/train/cam_{str(cam_num)}/'
    
    #기존 csv 호출
    cam_file_list_mp4 = glob(cam_dir + '*.mp4', recursive=True)
    print(cam_file_list_mp4) # 0번은 동영상, 1번은 csv


    #영상속 프레임을 프레임 단위로 추출
    save_dir = f'data/cam_{str(cam_num)}/'
    video = imageio.get_reader(cam_file_list_mp4[0], 'ffmpeg')
    for index, frame in tqdm(enumerate(video)):
        cv2.imwrite(save_dir +str(index)+'.jpg', cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
    print(f'Done!')

In [4]:
!ls /home/workspace/data/.train/.task154/test/cam_0

cam_0.mp4


In [6]:
# test 

for i in range(0,6):
    print(f'Start number : {i}')
    print('-------------------------')
    cam_num = i # 0부터 5까지 있다
    
    #추후 경로 변경 시수정이 필요한 사항 
    cam_dir = f'/home/workspace/data/.train/.task154/test/cam_{str(cam_num)}/'
    
    #기존 csv 호출
    cam_file_list_mp4 = glob(cam_dir + '*.mp4', recursive=True)
    print(cam_file_list_mp4) # 0번은 동영상


    #영상속 프레임을 프레임 단위로 추출
    save_dir = f'data_test/cam_{str(cam_num)}/'
    video = imageio.get_reader(cam_file_list_mp4[0], 'ffmpeg')
    for index, frame in tqdm(enumerate(video)):
        cv2.imwrite(save_dir +str(index)+'.jpg', cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
    print(f'Done!')
    print()

Start number : 0
-------------------------
['/home/workspace/data/.train/.task154/test/cam_0/cam_0.mp4']


4922it [03:35, 22.79it/s]
0it [00:00, ?it/s]

Done!

Start number : 1
-------------------------
['/home/workspace/data/.train/.task154/test/cam_1/cam_1.mp4']


4922it [03:50, 21.31it/s]


Done!

Start number : 2
-------------------------
['/home/workspace/data/.train/.task154/test/cam_2/cam_2.mp4']


...

Done!

Start number : 4
-------------------------
['/home/workspace/data/.train/.task154/test/cam_4/cam_4.mp4']


4922it [04:00, 20.46it/s]


Done!

Start number : 5
-------------------------
['/home/workspace/data/.train/.task154/test/cam_5/cam_5.mp4']


4922it [04:18, 19.05it/s]

Done!



## 2. CSV extract

In [ ]:
for i in range(0,6):
    cam_num = i # 0부터 5까지 있다
    print(f'Start number : {i}')
    
    #추후 경로 변경 시수정이 필요한 사항 
    cur_dir = '/home/workspace/user-workspace/object_tracking/madfalcon/'
    cam_dir = f'/home/workspace/data/.train/.task154/train/cam_{str(cam_num)}/'

    #기존 csv 호출
    cam_file_list_csv = glob(cam_dir + '*.csv', recursive=True)
    save_dir = f'data/cam_{str(cam_num)}/'
#     print(cam_file_list_csv)

    
    #데이터 프레임 호출
    df = pd.read_csv(cam_file_list_csv[0])
    df['width'] = 1920
    df['height'] = 1080
    
    df = df[['frame_no_cam', 'width', 'height', 'person_id', 'x_top_left_BB', 'y_top_left_BB', 'x_bottom_right_BB', 'y_bottom_right_BB']]
    df.columns = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    
    for i in tqdm(range(len(df))):
        tmp_str = cur_dir + save_dir + str(df['filename'][i]) + '.jpg'
        df['filename'][i] = tmp_str
    num = df._get_numeric_data()
    num[num < 0] = 0
    
    #csv파일 저장
    df.to_csv(save_dir + f'cam_{str(cam_num)}_labels.csv', index=None)
    print('done!')

In [ ]:
# csv file check
for i in range(6):
    cam_num = i # 0부터 5까지 있다
    save_dir = f'data/cam_{str(cam_num)}/'
    cam_file_list_csv = glob(save_dir + '*.csv', recursive=True)
    df = pd.read_csv(cam_file_list_csv[0])
    display(df.head(3))

## 3. Make label.pbtxt

In [ ]:
for i in range(0,6):
    cam_num = i # 0부터 5까지 있다
    print(f'Start number : {i}')
    
    #추후 경로 변경 시수정이 필요한 사항 
    save_dir = f'data/cam_{str(cam_num)}/'
    cam_dir = f'/home/workspace/data/.train/.task154/train/cam_{str(cam_num)}/'
    output_name = f'cam_{str(cam_num)}_labelmap.pbtxt'
    
    cam_file_list_csv = glob(save_dir + '*.csv', recursive=True)
    df = pd.read_csv(cam_file_list_csv[0])
    categories = sorted(list(df['class'].unique()))
    
    end = '\n'
    s = ' '
    class_map = {}
    for ID, name in tqdm(enumerate(categories)):
        out = ''
        out += 'item' + s + '{' + end
        out += s*2 + 'id:' + ' ' + (str(ID+1)) + end
        out += s*2 + 'name:' + ' ' + '\'' + str(name) + '\'' + end
        out += '}' + end*2


        with open(save_dir + output_name, 'a') as f:
            f.write(out)

        class_map[name] = ID+1
    
    print('done!')

## 4. Make TFrecord

In [1]:
#protocol buffer compile
!protoc /home/workspace/user-workspace/object_tracking/madfalcon/tf_models/research/object_detection/protos/*.proto --python_out=.

/home/workspace/user-workspace/object_tracking/madfalcon/tf_models/research/object_detection/protos/anchor_generator.proto: File does not reside within any path specified using --proto_path (or -I).  You must specify a --proto_path which encompasses this file.  Note that the proto_path must be an exact prefix of the .proto file names -- protoc is too dumb to figure out when two paths (e.g. absolute and relative) are equivalent (it's harder than you think).


In [2]:
for i in range(6):
    cam_num = i
    print(f'Start number : {i}')
    print('-------------------------')
    #추후 경로 변경 시수정이 필요한 사항 
    full_img_dir = f'/home/workspace/user-workspace/object_tracking/madfalcon/data/cam_{cam_num}/'
    full_csv_dir = f'/home/workspace/user-workspace/object_tracking/madfalcon/data/cam_{cam_num}/cam_{cam_num}_labels.csv'
    full_labelmap_dir = f'/home/workspace/user-workspace/object_tracking/madfalcon/data/cam_{cam_num}/cam_{cam_num}_labelmap.pbtxt'
    full_output_dir = f'/home/workspace/user-workspace/object_tracking/madfalcon/data/cam_{cam_num}/cam_{cam_num}_train.record'
    
    !python '/home/workspace/user-workspace/object_tracking/madfalcon/tf_models/research/20201104_custom_create_tfrecord.py' \
        -C $full_csv_dir \
        -l $full_labelmap_dir \
        -i $full_img_dir \
        -o $full_output_dir
    
    print('done!')
    print()

Start number : 0
-------------------------
label_map_list
{'1': 1, '2': 2, '3': 3, '5': 4, '7': 5, '8': 6, '9': 7, '11': 8, '12': 9, '14': 10, '15': 11, '16': 12, '19': 13, '20': 14, '25': 15, '28': 16, '38': 17, '41': 18, '43': 19, '44': 20, '46': 21, '49': 22, '55': 23, '59': 24, '149': 25, '182': 26, '189': 27, '194': 28, '199': 29, '207': 30, '213': 31}
Successfully created the TFRecord file: /home/workspace/user-workspace/object_tracking/madfalcon/data/cam_0/cam_0_train.record
done!

Start number : 1
-------------------------
label_map_list
{'0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '14': 13, '15': 14, '16': 15, '19': 16, '20': 17, '22': 18, '24': 19, '25': 20, '27': 21, '28': 22, '29': 23, '30': 24, '31': 25, '35': 26, '38': 27, '41': 28, '42': 29, '43': 30, '44': 31, '46': 32, '49': 33, '55': 34, '56': 35, '59': 36, '66': 37, '75': 38, '85': 39, '87': 40, '88': 41, '96': 42, '106': 43, '110': 44, '115': 45, '122': 46, '